In [1]:
# importing libraries.

import my_utils

import html
import time
import functools
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as sw
import string
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression




In [2]:
# reading the dataset.

#df=pd.read_csv('data.tsv', sep='\t', header=0, error_bad_lines=False)
tw=pd.read_csv('Twitter Sentiments.csv', sep='\t', header=0, error_bad_lines=False)

C:\Users\Sawan Kumar\AppData\Local\Temp\ipykernel_5824\304028001.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  tw=pd.read_csv('Twitter Sentiments.csv', sep='\t', header=0, error_bad_lines=False)


In [3]:
tw

,"id,label,tweet"
0,"1,0, @user when a father is dysfunctional and ..."
1,"2,0,@user @user thanks for #lyft credit i can'..."
2,"3,0, bihday your majesty"
3,"4,0,#model i love u take with u all the time..."
4,"5,0, factsguide: society now #motivation"
...,...
31957,"31958,0,ate @user isz that youuu?ðððð..."
31958,"31959,0, to see nina turner on the airwaves t..."
31959,"31960,0,listening to sad songs on a monday mor..."
31960,"31961,1,""@user #sikh #temple vandalised in in ..."


In [14]:
temp=tw.head(10)
temp


,"id,label,tweet"
0,"1,0, @user when a father is dysfunctional and ..."
1,"2,0,@user @user thanks for #lyft credit i can'..."
2,"3,0, bihday your majesty"
3,"4,0,#model i love u take with u all the time..."
4,"5,0, factsguide: society now #motivation"
5,"6,0,[2/2] huge fan fare and big talking before..."
6,"7,0, @user camping tomorrow @user @user @user ..."
7,"8,0,the next school year is the year for exams..."
8,"9,0,we won!!! love the land!!! #allin #cavs #c..."
9,"10,0, @user @user welcome here ! i'm it's s..."


In [ ]:
# We require only the review_body, star_rating columns which describes the reviews, star rating of each review respectively.

In [13]:
tw=tw[['tweet']]

KeyError: "None of [Index(['tweet'], dtype='object')] are in the [columns]"

In [10]:
#Removing the NULL, missing values and reset the index.

In [10]:
df=df.dropna()
df = df.reset_index(drop=True)
df

,review_body,star_rating
0,so beautiful even tho clearly not high end ......,5
1,"Great product.. I got this set for my mother, ...",5
2,Exactly as pictured and my daughter's friend l...,5
3,Love it. Fits great. Super comfortable and nea...,5
4,Got this as a Mother's Day gift for my Mom and...,5
...,...,...
1766743,It is nice looking and everything (it is sterl...,4
1766744,"my boyfriend bought me this last christmas, an...",4
1766745,This is a great way to quickly start learning ...,4
1766746,the 14kt gold earrings look remarkable...would...,5


In [9]:
#Labelling Reviews: 
#The reviews with star rating 4,5 are labelled as positive reviews and 
#1,2,3 are labelled as negative reviews.

In [11]:
# labelling and adding the new column to the dataset.

df['star_rating']=df['star_rating'].astype(int) #convert the star_rating column to int
df['pos_neg']=np.where(df['star_rating']>=4,'pos','neg') 

In [12]:
df.head(10)

,review_body,star_rating,pos_neg
0,so beautiful even tho clearly not high end ......,5,pos
1,"Great product.. I got this set for my mother, ...",5,pos
2,Exactly as pictured and my daughter's friend l...,5,pos
3,Love it. Fits great. Super comfortable and nea...,5,pos
4,Got this as a Mother's Day gift for my Mom and...,5,pos
5,I Love It Make Me Won't To Get Another,5,pos
6,"True to size, unique",5,pos
7,This necklace is BEAUTIFUL......is a great acc...,5,pos
8,just perfect,5,pos
9,Love it !,5,pos


In [13]:
# now considering only 2 columns
df=df[['pos_neg','review_body']]
df.head()

,pos_neg,review_body
0,pos,so beautiful even tho clearly not high end ......
1,pos,"Great product.. I got this set for my mother, ..."
2,pos,Exactly as pictured and my daughter's friend l...
3,pos,Love it. Fits great. Super comfortable and nea...
4,pos,Got this as a Mother's Day gift for my Mom and...


In [14]:
df.isnull().sum()

pos_neg        0
review_body    0
dtype: int64

In [15]:
df.shape

(1766748, 2)

In [16]:
df.pos_neg.value_counts()

pos    1351295
neg     415453
Name: pos_neg, dtype: int64

In [17]:
# Sample positive reveiws to get a balanced dataset
neg = df.loc[df.pos_neg=='neg']
pos = df.loc[df.pos_neg=='pos'].sample(n=df.pos_neg.value_counts()['neg'], random_state=42)

In [18]:
print(type(pos))
print("pos:", len(pos), ", neg:", len(neg))

<class 'pandas.core.frame.DataFrame'>
pos: 415453 , neg: 415453


In [19]:
df


,pos_neg,review_body
0,pos,so beautiful even tho clearly not high end ......
1,pos,"Great product.. I got this set for my mother, ..."
2,pos,Exactly as pictured and my daughter's friend l...
3,pos,Love it. Fits great. Super comfortable and nea...
4,pos,Got this as a Mother's Day gift for my Mom and...
...,...,...
1766743,pos,It is nice looking and everything (it is sterl...
1766744,pos,"my boyfriend bought me this last christmas, an..."
1766745,pos,This is a great way to quickly start learning ...
1766746,pos,the 14kt gold earrings look remarkable...would...


In [25]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Sawan Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Sawan Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basqu

[nltk_data]    | Downloading package nombank.1.0 to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\Sawan Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nonbreaking_prefixes.zip.
[nltk_data]    | Downloading package nps_chat to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nps_chat.zip.
[nltk_data]    | Downloading package omw to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package omw-1.4 to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package opinion_lexicon to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\opinion_lexicon.zip.
[nltk_data]    | Downloading package panlex_swadesh to C:\Users\Sawan
[nltk_data]    |     K

[nltk_data]    | Downloading package vader_lexicon to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package verbnet to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\verbnet.zip.
[nltk_data]    | Downloading package verbnet3 to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\verbnet3.zip.
[nltk_data]    | Downloading package webtext to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\webtext.zip.
[nltk_data]    | Downloading package wmt15_eval to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\wmt15_eval.zip.
[nltk_data]    | Downloading package word2vec_sample to C:\Users\Sawan
[nltk_data]    |     Kumar\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping models\word2vec_sample.zip.
[nltk_da

True

In [27]:
# define the preprocessing function 

lemmatizer = nltk.WordNetLemmatizer()
stopwords = sw.words('english')
stopwords = stopwords + ['not_' + w for w in stopwords]

# transform punctuation to blanks
trans_punct = str.maketrans(string.punctuation,' '*len(string.punctuation)) 

# pad punctuation with blanks
pad_punct = str.maketrans({key: " {0} ".format(key) for key in string.punctuation}) 
# remove "_" from string.punctuation
invalidChars = str(string.punctuation.replace("_", ""))  

In [28]:
def preprocessing(line, ngram=1, neg_handling=True, remove_stop=False):
    """
    Preprocessing the review texts
    @params:
        line                       - Required: the input text (Str)
        ngram                  - Optional: number n in the n-gram model(Int, 1, 2, or 3)
        neg_handling       - Optional: whether to perform negation handling (Boolean)
        remove_stop        -Optional: whether to remove the stop words (Boolean)
    """
        
    line = html.unescape(str(line))
    line = str(line).replace("can't", "can not")
    line = str(line).replace("n't", " not")
    
    if neg_handling:
        line = str(line).translate(pad_punct)  # If performing negation handling, pad punctuations with blanks
        line = nltk.word_tokenize(line.lower()) # Word normalization and tokenization
        tokens = []
        negated = False
        for t in line:
            if t in ['not', 'no']:
                negated = not negated
            elif t in string.punctuation or not t.isalpha():
                negated = False
            else:
                tokens.append('not_' + t if negated else t)  # add "not_" prefix to words behind "not", or "no"     
    else:
        line = str(line).translate(trans_punct)  # If not performing negation handling, remove punctuations
        line = nltk.word_tokenize(line.lower()) # Word normalization and tokenization
        tokens = line
    
    if ngram==2:
        bi_tokens = list(nltk.bigrams(line))
        bi_tokens = list(map('_'.join, bi_tokens))
        bi_tokens = [i for i in bi_tokens if all(j not in invalidChars for j in i)]
        tokens = tokens + bi_tokens

    if ngram==3:
        bi_tokens = list(nltk.bigrams(line))
        bi_tokens = list(map('_'.join, bi_tokens))
        bi_tokens = [i for i in bi_tokens if all(j not in invalidChars for j in i)]
        tri_tokens = list(nltk.trigrams(line))
        tri_tokens = list(map('_'.join, tri_tokens))
        tri_tokens = [i for i in tri_tokens if all(j not in invalidChars for j in i)]
        tokens = tokens + bi_tokens + tri_tokens    
     
    if remove_stop:
        line = [lemmatizer.lemmatize(t) for t in tokens if t not in stopwords]
    else:
        line = [lemmatizer.lemmatize(t) for t in tokens] 
    
    return ' '.join(line)

In [29]:

line = "I don't think this book has any decent information!!! It is full of typos and factual errors that I can't ignore."

preprocessing(line, ngram=1, neg_handling=False, remove_stop=False)

'i do not think this book ha any decent information it is full of typo and factual error that i can not ignore'

In [30]:
preprocessing(line, ngram=1, neg_handling=False, remove_stop=True)


'think book decent information full typo factual error ignore'

In [31]:
preprocessing(line, ngram=1, neg_handling=True, remove_stop=False)

'i do not_think not_this not_book not_has not_any not_decent not_information it is full of typo and factual error that i can not_ignore'

In [32]:
preprocessing(line, ngram=3, neg_handling=True, remove_stop=False)


'i do not_think not_this not_book not_has not_any not_decent not_information it is full of typo and factual error that i can not_ignore i_do do_not not_think think_this this_book book_has has_any any_decent decent_information it_is is_full full_of of_typos typos_and and_factual factual_errors errors_that that_i i_can can_not not_ignore i_do_not do_not_think not_think_this think_this_book this_book_has book_has_any has_any_decent any_decent_information it_is_full is_full_of full_of_typos of_typos_and typos_and_factual and_factual_errors factual_errors_that errors_that_i that_i_can i_can_not can_not_ignore'

In [33]:
#  preprocessing the data
# Perform data preprocessing with negation handling, tri-gram modeling, without removing the stop words

# preprocessing the positive reviews

pos_data = []
n_pos = len(pos)
for i, p in enumerate(pos['review_body']):
    pos_data.append(preprocessing(p, ngram=3))
    my_utils.print_progress(bar_length=50, decimals=0, iteration=i+1, total=n_pos, prefix='Preprocessing pos data: ')


Preprocessing pos data:  |==================================================| 100% 


In [34]:
# preprocessing the negative reviews

neg_data = []
n_neg = len(neg)
for i, n in enumerate(neg['review_body']):
    neg_data.append(preprocessing(n, ngram=3))
    my_utils.print_progress(bar_length=50, decimals=0, iteration=i+1, total=n_neg, prefix='Preprocessing neg data: ')

Preprocessing neg data:  |==================================================| 100% 


In [ ]:
# Combine the preprocessed data
data = pos_data + neg_data
labels = np.concatenate((pos['pos_neg'].values, neg['pos_neg'].values))

In [ ]:
 # 2.3 Split to training, validation, and test sets
# split the dataset to training, validation, test sets by 60-20-20
train_data, rest_data, train_labels, rest_labels = train_test_split(data, labels, test_size=0.4, 
                                                                    stratify=labels, random_state=1234)
valid_data, test_data, valid_labels, test_labels = train_test_split(rest_data, rest_labels, test_size=0.5, 
                                                                    stratify=rest_labels, random_state=1234)
print("training size = ", len(train_data), "validation size = ", len(valid_data), "testing size = ", len(test_data))

In [ ]:
# 3. Feature Extraction

# 3.1 Compute the frequency of words

# Push all tokens and compute the frequency of words
tokens = [word for line in train_data for word in nltk.word_tokenize(line)]
word_features = nltk.FreqDist(tokens)

In [ ]:
print(word_features)

In [ ]:
# Print the 10 most common words
word_features.most_common(10)

In [ ]:
# Remove features (words) which occur only once (This is to be used in the basic modeling process)
topwords = [fpair[0] for fpair in list(word_features.most_common(len(word_features))) if fpair[1]>=2] 
len(topwords) 

In [ ]:
# 3.2 Vectorizer and Tf-idf term weighting
# Convert a collection of raw documents to a matrix of TF-IDF features.
# Equivalent to CountVectorizer followed by TfidfTransformer.
tf_vec = TfidfVectorizer()

tf_vec.fit_transform([' '.join(topwords)])

In [ ]:
# 3.3 Feature Extraction

# Extract features from training set
# Vocabulary is from topwords
train_features = tf_vec.transform(train_data)

In [ ]:
# Array[n_train_data * n_features]
train_features.shape

In [ ]:
# Extract features from test set
test_features = tf_vec.transform(test_data)

In [ ]:
test_features.shape

In [ ]:
# 4.1 Multinomial Naive Bayes Classification Model (MultinomialNB)
mnb_model = MultinomialNB()
mnb_model

In [ ]:
# Train Model
mnb_model.fit(train_features, train_labels)

In [ ]:
# Predict
pred = mnb_model.predict(test_features)
print(pred)

In [ ]:
# Metrics
accuracy = metrics.accuracy_score(test_labels, pred)
print(accuracy)

In [ ]:

print(metrics.classification_report(y_true=test_labels, y_pred=pred, digits=4))

In [ ]:
# 4.2 Logistic Regression Mode (Logistic Regression)
lgr_model = LogisticRegression()
print(lgr_model, end='\n'*2)


lgr_model.fit(train_features, train_labels)
lgr_pred = lgr_model.predict(test_features)

print('Accuracy = %.5f' % metrics.accuracy_score(test_labels, lgr_pred))
print(metrics.classification_report(y_pred=lgr_pred, y_true=test_labels, digits=4))